In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns   

from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import tree

In [ ]:
df_train = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Dataset/titanic/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Dataset/titanic/test.csv")

In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_train.shape

(891, 12)

In [ ]:
df_train.Cabin.isnull().sum()

687

In [ ]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df_train["Pclass"] = df_train.Pclass.astype("object")
df_train["Sex"] = df_train.Sex.astype("category")
df_train["Ticket"] = df_train.Ticket.astype("category")
df_train["Survived"] = df_train.Survived.astype("category")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    object  
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    category
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    object  
dtypes: category(3), float64(2), int64(3), object(4)
memory usage: 88.0+ KB


In [ ]:
df_train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [ ]:
df_train.isnull().values.sum()

866

In [ ]:
sns.heatmap(df_train.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
X = df_train.dropna(axis = 0)
y = X.Survived
X = X.drop(["Survived", 'PassengerId', 'Name', "Cabin", "Ticket"], axis = 1)
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 

Without any other tools, use only LC

In [ ]:
clf =  LogisticRegression(random_state=0).fit(X_train, y_train) 

y_predict = clf.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[ 6  6]
 [ 6 19]]
              precision    recall  f1-score   support

           0       0.50      0.50      0.50        12
           1       0.76      0.76      0.76        25

    accuracy                           0.68        37
   macro avg       0.63      0.63      0.63        37
weighted avg       0.68      0.68      0.68        37



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
812,2,male,35.0,0,0,28206,10.5000,NaN,S
641,1,female,24.0,0,0,PC 17477,69.3000,B35,C
220,3,male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S
65,3,male,NaN,1,1,2661,15.2458,NaN,C
332,1,male,38.0,0,1,PC 17582,153.4625,C91,S


Without any tools, use only DT

In [ ]:
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("DT") 
graph

In [ ]:
y_predict = clf_tree.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[ 8  4]
 [ 1 24]]
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.86      0.96      0.91        25

    accuracy                           0.86        37
   macro avg       0.87      0.81      0.83        37
weighted avg       0.87      0.86      0.86        37



Let's use Emsemble

In [ ]:
# Bagging with using DT == RT
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier 

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1, oob_score = True)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
y_predict = bag_clf.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[ 4  4]
 [ 2 27]]
              precision    recall  f1-score   support

           0       0.67      0.50      0.57         8
           1       0.87      0.93      0.90        29

    accuracy                           0.84        37
   macro avg       0.77      0.72      0.74        37
weighted avg       0.83      0.84      0.83        37



In [ ]:
bag_clf.oob_score_

0.7054794520547946

In [ ]:
bag_clf.oob_decision_function_

In [ ]:
# RT 
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_red_rf = rnd_clf.predict(X_test)

In [ ]:
y_predict = rnd_clf.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[ 4  4]
 [ 1 28]]
              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.88      0.97      0.92        29

    accuracy                           0.86        37
   macro avg       0.84      0.73      0.77        37
weighted avg       0.86      0.86      0.85        37



In [ ]:
for name, score in zip(X_train.columns, rnd_clf.feature_importances_):
  print(name, score)


Age 0.23166793278138442
SibSp 0.040473240227602056
Parch 0.039306005458240284
Fare 0.22008039012693045
Pclass_1 0.013578365359699772
Pclass_2 0.011593529602913709
Pclass_3 0.017631225959602093
Sex_female 0.18646015074706587
Sex_male 0.19916606310269158
Embarked_C 0.020335714673963265
Embarked_Q 0.0035343197638198548
Embarked_S 0.016173062196086475


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 1), n_estimators = 500, 
                             algorithm = "SAMME.R", learning = 0.5)
ada_clf.fit(X_train, y_train)

y_red_rf = ada_clf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict)) 

[[ 4  4]
 [ 1 28]]
              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.88      0.97      0.92        29

    accuracy                           0.86        37
   macro avg       0.84      0.73      0.77        37
weighted avg       0.86      0.86      0.85        37

